In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from decimal import Decimal

# Configuration
PRECISION = Decimal("1e18")  # 10^18 precision as used in Vyper
NUM_STEPS = 3_00


# Function to update xcp_profit as in the Curve contract
def update_xcp_profit(old_xcp_profit, virtual_price, old_virtual_price):
    return old_xcp_profit * virtual_price / old_virtual_price


# Generate a virtual price series with random fluctuations
def main():
    # Initialize both metrics at 10^18 (1.0)
    virtual_prices = [PRECISION]
    xcp_profits = [PRECISION]
    xcp_profit_a = PRECISION
    # Generate random virtual price changes
    for i in range(1, NUM_STEPS):
        # Admin fee claim (LP+DAO)
        fee_claim_flag = bool(i % 15 == 0)
        if fee_claim_flag:
            crude_profit = xcp_profits[-1] - xcp_profit_a
            if crude_profit > 0:
                fee_amount = Decimal(int(crude_profit * Decimal(0.01) / 2))
                xcp_profits[-1] -= fee_amount
                xcp_profit_a = xcp_profits[-1]
                virtual_prices[-1] -= fee_amount
                print(f"fee claim {i}, {fee_amount}")
            # skip rest of the loop
            continue

        # Random change between 0% and +0.15% (slightly biased toward growth)
        # emulate swap -> fees boost virtual price
        change_pct = Decimal(np.random.uniform(0.00, 0.0015))
        # Calculate new virtual price (simplified from contract)
        new_vp = virtual_prices[-1] * (Decimal("1") + change_pct)

        # Ensure integer arithmetic like in contract
        new_vp = Decimal(int(new_vp))

        # Calculate new xcp_profit
        new_xcp = update_xcp_profit(xcp_profits[-1], new_vp, virtual_prices[-1])

        # emulate tweak_price
        thr = Decimal(2e12)

        # Rebalance fee spend
        rebalance_flag = bool(i % 10 == 0)
        rebalance_buffer = PRECISION + (new_xcp - PRECISION) / 2
        if rebalance_flag and new_vp > rebalance_buffer + thr:
            change_pct = Decimal(np.random.uniform(-0.005, -0.001))
            # Calculate new virtual price (simplified from contract)
            # candidate_vp = virtual_prices[-1] * (Decimal('1') + change_pct)
            candidate_vp = rebalance_buffer + thr
            candidate_vp = Decimal(int(candidate_vp))
            if candidate_vp >= max(rebalance_buffer, PRECISION):
                new_vp = candidate_vp
                print(f"rebalance {i}, {new_vp}")
        virtual_prices.append(new_vp)
        xcp_profits.append(new_xcp)

    # Convert to regular floats for plotting (normalize to 1.0)
    vp_display = [float(v / PRECISION) for v in virtual_prices]
    xcp_display = [float(x / PRECISION) for x in xcp_profits]
    xcp_half = [float((PRECISION + (x - PRECISION) / 2) / PRECISION) for x in xcp_profits]
    xcp_sqrt = [float(np.sqrt(x / PRECISION)) for x in xcp_profits]
    # Calculate the ratio between them
    ratios = [xcp / vp for xcp, vp in zip(xcp_display, vp_display)]

    # Visualization
    plt.figure(figsize=(10, 8))

    # Plot values
    plt.subplot(2, 1, 1)
    plt.plot(vp_display, label="Virtual Price", linewidth=2)
    plt.plot(xcp_display, label="XCP Profit", linewidth=2, linestyle="--")
    plt.plot(xcp_half, label="XCP Half", linewidth=2, linestyle=":")
    plt.plot(xcp_sqrt, label="XCP Sqrt", linewidth=2, linestyle=":")
    plt.title("Virtual Price and XCP Profit Over Time")
    plt.ylabel("Value")
    plt.legend()
    plt.grid(True)

    # Plot ratio
    plt.subplot(2, 1, 2)
    plt.plot(ratios, label="XCP Profit / Virtual Price")
    plt.axhline(y=1.0, color="r", linestyle="--")
    plt.title("Ratio of XCP Profit to Virtual Price")
    plt.ylabel("Ratio")
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    # Print basic statistics
    print(f"Final VP: {vp_display[-1]:.6f}, Final XCP: {xcp_display[-1]:.6f}")
    print(f"Are VP and XCP proportional? {all(abs(r - 1.0) < 1e-10 for r in ratios)}")
    print(f"Max deviation from 1.0: {max(abs(r - 1.0) for r in ratios):.10f}")


if __name__ == "__main__":
    main()

In [ ]:
r = 0.0015
n = 3000

init = 1

res = (1 + r / 2) ** n

print(res)

In [ ]:
wad = 10**18
pool_value_init = 3000000000000000000000000
pool_value_post = 6019243555149172634124923
total_fees = pool_value_post - pool_value_init
vp_0 = 1000000000000000000
vp_1 = 2006414292517988114
xcp_profit_0 = 1000000000000000000
xcp_profit_1 = 2006414292517988114
lp_supply = 38729833462074168852585
D_0 = 3000000000000000000000000
D_1 = 6009324343410821147499040
# fees: uint256 = unsafe_div(
#     unsafe_sub(xcp_profit, xcp_profit_a) * admin_fee=admin_fee, 2 * 10**10
# )

fees = (xcp_profit_1 - xcp_profit_0) // 2 // 2 * 2
print(f"fees: {fees}")
frac = vp_1 * 10**18 // (vp_1 - fees) - 10**18
print(f"frac: {frac}")
admin_share = lp_supply * frac // 10**18
print(f"admin_share: {admin_share}")
fees_rate = admin_share * 10**18 // (admin_share + lp_supply)
fees_value = fees_rate * pool_value_post // 10**18
print(f"fees value: {fees_value}")
print(f"pool_profits: {(pool_value_post - pool_value_init)}")
print(f"rate: {fees_value / (pool_value_post - pool_value_init)}")

In [ ]:
vp_1 / vp_0

In [ ]:
pool_value_post / pool_value_init

In [ ]:
pool_value_init

In [ ]:
pool_value_post

In [ ]:
D_1 / D_0

In [ ]:
class PoolModel:
    def __init__(
        self,
        admin_fee=0.5,
        rebalance_offset: float = 0.01,
        subtract_factor: float = 2.0,
        xcp_growth: float = 0.01,
        mode="linear",
    ):
        self.vp = 1.0
        self.xcp_profit = 1.0
        self.xcp_profit_a = 1.0
        self.admin_claimed_fees = 0.0
        self.offset = rebalance_offset
        self.total_supply = 1.0
        self.xcp = 1.0
        self.subtract_factor = subtract_factor
        self.xcp_growth = xcp_growth
        self.mode = mode
        self.admin_fee = admin_fee
        # bookkeeping for plotting
        self.step_count = 0
        self.history = {
            "vp": [],
            "xcp_profit": [],
            "xcp_profit_a": [],
            "reserved_xcp_profit": [],
            "admin_claimed_fees": [],
            "xcp": [],
        }
        self.claim_marks = []  # list of (step, vp)
        self.rebalance_marks = []  # list of (step, vp)

    # ---------------- core actions ---------------- #

    def trade(self, rebalance_flag=False) -> None:
        xcp_new = self.xcp * (1.0 + self.xcp_growth)
        # xcp_new = self.xcp + self.xcp_growth
        self.xcp = xcp_new
        self.tweak_price(rebalance_flag)

    def claim_fees(self) -> float:
        if self.mode == "linear":
            return self.claim_fees_linear()
        elif "sqrt" in self.mode:
            return self.claim_fees_sqrt()
        else:
            raise ValueError(f"Invalid mode: {self.mode}")

    def claim_fees_linear(self) -> float:
        delta = self.xcp_profit - self.xcp_profit_a
        if delta <= 0.0:
            return 0.0

        fee = delta / 2 * self.admin_fee

        frac = self.vp / (self.vp - fee) - 1
        admin_share = self.total_supply * frac

        vp = self.xcp / (self.total_supply + admin_share)
        if vp < 1.0:
            return 0.0
        self.vp = vp
        self.xcp = vp * self.total_supply
        self.xcp_profit -= self.subtract_factor * fee
        self.xcp_profit_a = self.xcp_profit

        self.claim_marks.append((self.step_count, self.vp))
        return fee

    def claim_fees_sqrt(self) -> float:
        fee = self.admin_unclaimed_fees()

        frac = self.vp / (self.vp - fee) - 1
        admin_share = self.total_supply * frac

        vp = self.xcp / (self.total_supply + admin_share)
        if vp < 1.0:
            return 0.0

        if self.mode == "sqrt":
            self.xcp_profit = (
                np.sqrt(self.xcp_profit) * (1 - self.admin_fee)
                + np.sqrt(self.xcp_profit_a) * self.admin_fee
            ) ** 2
        elif self.mode == "sqrt_lin":
            self.xcp_profit = (
                self.xcp_profit * (1 - self.admin_fee) + self.xcp_profit_a * self.admin_fee
            )
        elif self.mode == "sqrt_admin":
            self.admin_claimed_fees += fee

        self.vp = vp
        self.xcp = vp * self.total_supply
        self.xcp_profit_a = self.xcp_profit

        self.claim_marks.append((self.step_count, self.vp))
        return fee

    def admin_unclaimed_fees(self) -> float:
        root_P = np.sqrt(self.xcp_profit)
        root_P_a = np.sqrt(self.xcp_profit_a)
        admin_full_fees = (root_P - root_P_a) * self.admin_fee
        admin_unclaimed = admin_full_fees - self.admin_claimed_fees
        return max(0.0, admin_unclaimed)

    def tweak_price(self, rebalance_flag=False) -> float:
        vp_old = self.vp
        vp_new = self.xcp / self.total_supply
        if self.mode == "sqrt_admin":
            # we must compound as vp_new/vp_old
            # but we can't compound whole body of xcp_profit, because we remove admin_claimed_fees
            # so we must compound only the part that is not admin_claimed_fees
            self.xcp_profit += (self.xcp_profit - self.admin_claimed_fees) * (vp_new / vp_old - 1)
        else:
            self.xcp_profit *= vp_new / vp_old

        if rebalance_flag and vp_new > self.reserved_xcp_profit(self.xcp_profit) + self.offset:
            reb_vp = self.reserved_xcp_profit(self.xcp_profit)
            reb_xcp = reb_vp * self.total_supply
            self.xcp = reb_xcp
            self.vp = reb_vp
            self.rebalance_marks.append((self.step_count, self.vp))
        else:
            self.vp = vp_new
        return False

    def reserved_xcp_profit(self, xcp_profit) -> float:
        if self.mode == "linear":
            return 1 + (xcp_profit - 1) / 2
        elif self.mode == "sqrt_admin":
            return np.sqrt(xcp_profit) - self.admin_claimed_fees
        elif "sqrt" in self.mode:
            return np.sqrt(xcp_profit)

    # -------------- simulation helpers ------------- #

    def _record(self) -> None:
        self.history["vp"].append(self.vp)
        self.history["xcp_profit"].append(self.xcp_profit)
        self.history["xcp_profit_a"].append(self.xcp_profit_a)
        self.history["reserved_xcp_profit"].append(self.reserved_xcp_profit(self.xcp_profit))
        self.history["admin_claimed_fees"].append(self.admin_claimed_fees)
        self.history["xcp"].append(self.xcp)

    def step(self, claim_interval, rebalance_interval) -> None:
        rebalance_flag = self.step_count % rebalance_interval == 0 and self.step_count > 0
        self.trade(rebalance_flag)
        if self.step_count % claim_interval == 0 and self.step_count > 0:
            self.claim_fees()
        self._record()
        self.step_count += 1

    # -------------- convenience run --------------- #

    def run(
        self, n_steps: int = 200, claim_interval: int = 50, rebalance_interval: int = 10
    ) -> None:
        for _ in range(n_steps):
            self.step(claim_interval, rebalance_interval)

In [ ]:
claim_interval = 200
rebalance_interval = 1
xcp_growth = 0.0001
n_steps = 1000
offset = 0.03
admin_fee = 0.5
# Define models with different parameters
models = {
    # '-2fee, linear': PoolModel(admin_fee=admin_fee, rebalance_offset=offset, subtract_factor=2, xcp_growth=xcp_growth, mode='linear'),
    # '-1fee, linear': PoolModel(admin_fee=admin_fee, rebalance_offset=offset, subtract_factor=1, xcp_growth=xcp_growth, mode='linear'),
    "sqrt": PoolModel(
        admin_fee=admin_fee,
        rebalance_offset=offset,
        subtract_factor=2,
        xcp_growth=xcp_growth,
        mode="sqrt",
    ),
    # 'sqrt_lin': PoolModel(admin_fee=admin_fee, rebalance_offset=offset, subtract_factor=2, xcp_growth=xcp_growth, mode='sqrt_lin'),
    "sqrt_admin": PoolModel(
        admin_fee=admin_fee,
        rebalance_offset=offset,
        subtract_factor=2,
        xcp_growth=xcp_growth,
        mode="sqrt_admin",
    ),
    # PoolModel(subtract_factor=1, xcp_growth=xcp_growth, mode='linear'),
}
colors = [
    "tab:blue",
    "tab:orange",
    "tab:green",
    "tab:red",
    "tab:purple",
    "tab:brown",
    "tab:pink",
    "tab:gray",
    "tab:olive",
    "tab:cyan",
]
# Run all models
for _, model in models.items():
    model.run(n_steps=n_steps, claim_interval=claim_interval, rebalance_interval=rebalance_interval)

# Plot all on the same axes
plt.figure(figsize=(6, 4))
for i, (key, model) in enumerate(models.items()):
    clr = colors[i]
    lw = 1
    plt.plot(model.history["vp"], label=f"vp, {key}", color=clr, linewidth=lw)
    plt.plot(
        model.history["xcp_profit"],
        label=f"xcp_profit, {key}",
        linestyle="-.",
        color=clr,
        linewidth=lw,
    )
    # Optionally plot xcp_lp_share as well
    plt.plot(
        model.history["reserved_xcp_profit"],
        label=f"xcp_lp_share, {key}",
        linestyle=":",
        color=clr,
        linewidth=lw,
    )
    print(
        f"Model {i}, ({key}) has {len(model.rebalance_marks)} rebalances and {len(model.claim_marks)} claims"
    )

plt.title("Comparison of Pool Models")
plt.xlabel("step")
plt.ylabel("value")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
model